In [ ]:
import os
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'

import numpy as np
import pandas as pd
from tqdm import tqdm

from evaluation.environment import TrainingEnvironment

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# Official hyperparameters for this competition (do not modify)
N_TRAIN_USERS = 1000
N_TEST_USERS = 2000
N_ITEMS = 209527
HORIZON = 2000
TEST_EPISODES = 5
SLATE_SIZE = 5

In [ ]:
# Dataset paths
USER_DATA = os.path.join('dataset', 'user_data.json')
ITEM_DATA = os.path.join('dataset', 'item_data.json')

# Output file path
OUTPUT_PATH = os.path.join('output', 'output.csv')

In [ ]:
M_USERS = 2000
N_ITEMS = 209527
SEED = 0

In [ ]:
# df_ratings = pd.DataFrame(columns=['user_id', 'item_id', 'rating'])
df_ratings = pd.read_csv('./dataset/ratings.csv')
df_ratings

In [ ]:
class FunkSVDRecommender(tf.keras.Model):
    '''
    Simplified Funk-SVD recommender model
    '''
    def __init__(self, m_users: int, n_items: int, embedding_size: int, learning_rate: float):
        '''
        Constructor of the model
        '''
        super().__init__()
        self.m = m_users
        self.n = n_items
        self.k = embedding_size
        self.lr = learning_rate

        # user embeddings P
        self.P = tf.Variable(tf.keras.initializers.RandomNormal()(shape=(self.m, self.k)))

        # item embeddings Q
        self.Q = tf.Variable(tf.keras.initializers.RandomNormal()(shape=(self.n, self.k)))

        # optimizer
        self.optimizer = tf.optimizers.Adam(learning_rate=self.lr)

    @tf.function
    def call(self, user_ids: tf.Tensor, item_ids: tf.Tensor) -> tf.Tensor:
        '''
        Forward pass used in training and validating
        '''
        # dot product the user and item embeddings corresponding to the observed interaction pairs to produce predictions
        y_pred = tf.reduce_sum(tf.gather(self.P, indices=user_ids) * tf.gather(self.Q, indices=item_ids), axis=1)

        return y_pred

    @tf.function
    def compute_loss(self, y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        '''
        Compute the MSE loss of the model
        '''
        loss = tf.losses.mean_squared_error(y_true, y_pred)

        return loss

    @tf.function
    def train_step(self, data: tf.Tensor) -> tf.Tensor:
        '''
        Train the model with one batch
        data: batched user-item interactions
        each record in data is in the format ['user_id', 'item_id', 'rating']
        '''
        user_ids = tf.cast(data[:, 0], dtype=tf.int32)
        item_ids = tf.cast(data[:, 1], dtype=tf.int32)
        y_true = tf.cast(data[:, 2], dtype=tf.float32)

        # slate = self.eval_predict_onestep(user_ids)
        # print(slate)
        # compute loss
        with tf.GradientTape() as tape:
            y_pred = self(user_ids, item_ids)
            loss = self.compute_loss(y_true, y_pred)

        # compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    @tf.function
    def val_step(self, data: tf.Tensor) -> tf.Tensor:
        '''
        Validate the model with one batch
        data: batched user-item interactions
        each record in data is in the format ['user_id', 'item_id', 'rating']
        '''
        user_ids = tf.cast(data[:, 0], dtype=tf.int32)
        item_ids = tf.cast(data[:, 1], dtype=tf.int32)
        y_true = tf.cast(data[:, 2], dtype=tf.float32)

        # compute loss
        y_pred = self(user_ids, item_ids)
        loss = self.compute_loss(y_true, y_pred)

        return loss

    @tf.function
    def eval_predict_onestep(self, query: tf.Tensor) -> tf.Tensor:
        '''
        Retrieve and return the MovieIDs of the 5 recommended movies given a query
        You should return a tf.Tensor with shape=(5,)
        query will be a tf.Tensor with shape=(1,) and dtype=tf.int64
        query is the UserID of the query
        '''
        # dot product the selected user and all item embeddings to produce predictions
        user_id = tf.cast(query, tf.int32)
        y_pred = tf.reduce_sum(tf.gather(self.P, user_id) * self.Q, axis=1)

        # select the top 5 items with highest scores in y_pred
        y_top_5 = tf.math.top_k(y_pred, k=5).indices

        return y_top_5

In [ ]:
# hyperparameters
EMBEDDING_SIZE = 512
BATCH_SIZE = 128
N_EPOCHS = 50
LEARNING_RATE = 1e-5

In [ ]:
model = FunkSVDRecommender(m_users=N_TRAIN_USERS, n_items=N_ITEMS, embedding_size=EMBEDDING_SIZE, learning_rate=LEARNING_RATE)

In [ ]:
ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=model)
manager = tf.train.CheckpointManager(ckpt, './ckpts/FSVD', max_to_keep=10, checkpoint_name='fsvd')

In [ ]:
# Initialize the testing environment
test_env = TrainingEnvironment()
pre = df_ratings.shape[0]
cur = 0

# The item_ids here is for the random recommender
item_ids = [i for i in range(N_ITEMS)]

# Repeat the testing process for 5 times
for _ in range(100):
    # [TODO] Load your model weights here (in the beginning of each testing episode)
    # [TODO] Code for loading your model weights...
    ckpt.restore(manager.latest_checkpoint)

    # Start the testing process
    with tqdm(desc='Testing') as pbar:
        # Run as long as there exist some active users
        while test_env.has_next_state():
            # Get the current user id
            cur_user = test_env.get_state()
            
            # [TODO] Employ your recommendation policy to generate a slate of 5 distinct items
            # [TODO] Code for generating the recommended slate...
            # Here we provide a simple random implementation
            slate = model.eval_predict_onestep(cur_user)

            # Get the response of the slate from the environment
            clicked_id, in_environment = test_env.get_response(slate)

            # [TODO] Update your model here (optional)
            # [TODO] You can update your model at each step, or perform a batched update after some interval
            # [TODO] Code for updating your model...
            ratings_data = []
            # mask = (df_ratings['user_id'] == cur_user) & (df_ratings['item_id'] == clicked_id)
            if clicked_id != -1: # and not mask.empty:
                # print('click', cur_user, clicked_id)
                ratings_data.append({'user_id': cur_user, 'item_id': clicked_id, 'rating': 5.0})

            df = pd.DataFrame(ratings_data)
            df_ratings = pd.concat([df_ratings, df], ignore_index=True)
            
            # Update the progress indicator
            pbar.update(1)

    # Reset the testing environment
    test_env.reset()

    # [TODO] Delete or reset your model weights here (in the end of each testing episode)
    # [TODO] Code for deleting your model weights...
    ckpt.restore(manager.latest_checkpoint)
    print(df_ratings.shape[0])
    df_ratings = df_ratings.drop_duplicates(subset=['user_id', 'item_id'], keep='last', ignore_index=True)

# Generate a DataFrame to output the result in a .csv file
    cur = df_ratings.shape[0]
    print(pre, cur)
    pre = cur
# df_ratings

In [ ]:
df_ratings = df_ratings.sort_values(by=['user_id', 'item_id'])
df_ratings
df_ratings.to_csv('./dataset/ratings.csv', index=False)

In [ ]:
count_5_star_ratings = len(df_ratings[df_ratings['rating'] == 5.0])
print("Number of rows with rating = 5.0:", count_5_star_ratings)